## Handling Inconsistant data by cleaning it

### Drop Null : Missing Values

In [ ]:
"""
    how = all (or) any
    
    axis = 0 - row
           1 - column

    alternate : df.na.drop()
"""

# To drop the null value rows in any column
df.dropna()

# To drop the null value rows in specific column
df.dropna(subset=["col_name"])

# To drop
df.dropna(how = '?', axis = ?)

### Drop Duplicates : Duplicates

In [ ]:
"""
   subset = we can mention column names
"""

# To drop the duplicates based on all the columns
df.dropDuplicates()

# To drop the duplicates based on the specified columns
df.dropDuplicates(['column1', 'column2'])

### Fill : Empty or Unknown Values

In [ ]:
"""
    value = can be scalar (or) dictionary if different column should need different values

    subset = we can mention column names for the scalar value

    alternate : df.na.fill()
"""

# To replace all NaN values in the DataFrame with 0
df.fillna(0)

# To replace missing values in column1 with 0 and missing values in column2 with 'unknown'
df.fillna({'column1': 0, 'column2': 'unknown'})

# To replace missing values with 0 only in column1 and column2
df.fillna(0, subset=['column1', 'column2'])

# To fill
df.fillna(value = ?, subset = [])

### Replace

In [ ]:
"""
    to_replace: The value or list of values to replace [ single value, a list of values, or a dictionary ]
    
    value: The new value or list of new values to replace to_replace values with. [ should be same count as to_replace ]
   
    subset: Optional. List of columns to apply the replacement to. If not specified, it applies to all columns.
    
    method: Optional. The method to use for replacement (e.g., 'pad', 'ffill', 'bfill').

    alternate = df.na.replace()
"""

# Replacing the specified value to the new value
DataFrame.replace(to_replace=None, value=None, subset=None, method=None)

### Outliers

In [ ]:
"""
    Don't know what is it : Z-scores, IQR (Interquartile Range), or custom thresholds.

    Data which were beyond expectation and actual value.  We need to remove (or) transform this values to get correcct analytics/conclustion
"""

# Filter
df.filter((col("value") >= lower_bound) & (col("value") <= upper_bound))

# Caping (or) Normalizing value
df.withColumn("col_name", when(col("col_name") < lower_bound, lower_bound).when(col("value") > upper_bound, upper_bound).otherwise(col("col_name")))

# Imputation
df.withColumn("value", when((col("value") < lower_bound) | (col("value") > upper_bound), median_val).otherwise(col("value")))

### Non formated data like +91, $ and Male -> M

In [ ]:
# Necessary imports
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, regexp_replace, when

# Creating spark session object
spark = SparkSession.builder \
    .appName("Data Cleaning") \
    .getOrCreate()

# Creating the data variable using row function
data = [
    Row(phone="+91-1234567890", currency="$100", gender="Male"),
    Row(phone="+44-9876543210", currency="£200", gender="Female"),
    Row(phone="+1-5555555555", currency="€300", gender="Non-binary"),
]

# Converting the data to DataFrame
df = spark.createDataFrame(data)

# printing the schema and data
df.show(truncate = False)
df.printSchema()

# Removing non-numeric characters from the phone number
df = df.withColumn("phone", regexp_replace(col("phone"), "[^0-9-]", "")) 

# Removing currency symbols
df = df.withColumn("currency", regexp_replace(col("currency"), "[$£€]", ""))

# Mapping gender to abbreviated form
df = df.withColumn("gender", when(col("gender") == "Male", "M")
                      .when(col("gender") == "Female", "F")
                      .otherwise("O"))

df.show()

# Stop the spark session
spark.stop()

### Invalid Format and Data (or) Data Anomalies

In [ ]:
# Change the mentioned column value to the desired format and create new column
df.withColumn("new_col_name", to_date(col("col_name"), "yyyy-MM-dd"))

df.withColumn("standard_date", 
    when(
        col("date").rlike(r"\d{4}-\d{2}-\d{2}"), to_date(col("date"), "yyyy-MM-dd")
    ).when(
        col("date").rlike(r"\d{2}/\d{2}/\d{4}"), to_date(col("date"), "dd/MM/yyyy")
    ).otherwise(
        to_date(col("date"), "yyyy.MM.dd")
    )
)

# Normalize number format by removing thousands separators
df = df.withColumn(
    "normalized_number",
    regexp_replace(col("number"), r"[^\d.]", "")  # Remove non-digit characters except decimal point
)

# Normalize to lowercase
df.withColumn("normalized_response", lower(col("response")))

"""
    Invalid data can be handled using UDF or replacing the value with when and otherwise
"""

### Type Casting

In [ ]:
# Cast the datatype of col_name to new data type
df = df.withColumn("col_name", col("col_name").cast("new_data_type"))

### When and Otherwise

In [ ]:
df = df.withColumn(
    "age_category",
    when(col("age") < 18, "Minor")
    .when(col("age").between(18, 65), "Adult")
    .otherwise("Senior")
)

### Filter

In [ ]:
"""
    \\d matches any digit.
    \\D matches any non-digit.
    \\w matches any word character (alphanumeric plus underscore).
    \\W matches any non-word character.
    \\s matches any whitespace character.
    \\S matches any non-whitespace character.
"""

# Just normal like as MySQL
df.filter(col("code").like("abc%"))

# Filter rows where the column contains digits using regular expression like
df.filter(col("col_name").rlike("\\d"))

### Use UDF to clean Data

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def clean_date(date_str):
    # Custom logic to handle different date formats
    # Example: Handle 'MM/DD/YYYY' and 'YYYY-MM-DD'
    # Return a standardized date string
    return date_str  # Placeholder for actual logic

clean_date_udf = udf(clean_date, StringType())

df = df.withColumn("cleaned_date", clean_date_udf(col("date")))

df.show()

### Adding and Renaming and Removing columns

In [ ]:
"""
    Adding Columns: Use withColumn.
    
    Renaming Columns: Use withColumnRenamed.
    
    Removing Columns: Use drop.
"""

# Adding a new column based on existing columns withcolumn(new_col_name, some_condition)
df.withColumn("new_col_name", col("old_some_col") + 1)  

# Renaming the column
df.withColumnRenamed("old_col_name", "new_col_name_for_same_col")

# Dropping existing column
df.drop("new_column")

## Other Topics :

In [ ]:
"""
    1. Logging

    2. Monitoring
"""